In [ ]:
from nltk.metrics import edit_distance
from sklearn.preprocessing import StandardScaler

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')


df_train = pd.read_csv("../input/train.csv.zip", encoding="ISO-8859-1")
df_test = pd.read_csv("../input/test.csv.zip", encoding="ISO-8859-1")
attribute_data = pd.read_csv('../input/attributes.csv.zip')
df_pro_desc = pd.read_csv('../input/product_descriptions.csv.zip')



df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)

num_train = df_train.shape[0]

# def str_stemmer(s):
#     return " ".join([stemmer.stem(word) for word in s.lower().split()])

def str_common_word(str1, str2):
    return sum(int(str2.find(word)>=0) for word in str1.split())


df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True, sort=False)

df_all = pd.merge(df_all, df_pro_desc, how='left', on='product_uid')





text cleaning

In [2]:
import string
import re
import nltk


ps = nltk.stem.PorterStemmer()
wn = nltk.WordNetLemmatizer()

stopwords = nltk.corpus.stopwords.words('english')

def remove_punctuation(text):
    txt = ''.join(character for character in text if character not in string.punctuation)
    return txt

def tokenization(text, stopwords = None):
    tokens = re.split('\W+', text)
    if stopwords:
        tokens = [token for token in tokens if token not in stopwords]
    return tokens
        
    
def stemming(tokens):
    new_tokens = list(map(ps.stem, tokens))
#     new_tokens = [ps.stem(token) for token in tokens]
    return new_tokens

def lemmatizing(tokens):
    new_tokens = list(map(wn.lemmatize, tokens))
    return new_tokens

In [3]:
from textblob.sentiments import NaiveBayesAnalyzer
from textblob import Blobber
classifier = Blobber(analyzer=NaiveBayesAnalyzer())


def spell_correction(word):
    return str(classifier(word).correct())
    

def correct_misspell(phrase):
    phrase = remove_punctuation(phrase)
    phrase = tokenization(phrase)
    phrase = ' '.join(map(spell_correction, phrase))
    return phrase

In [ ]:
df_all['typo_corrected'] = df_all['product_description'].map(spell_correction)

In [ ]:
df_all['product_title'] = df_all['product_title'].map(correct_misspell)

In [ ]:
df_all['search_term'] = df_all['search_term'].map(correct_misspell)

In [ ]:
pd.DataFrame(df_all).to_csv('hugecomputing_time.csv',index=False)

In [ ]:
from nltk.corpus import brown, stopwords
from nltk.cluster.util import cosine_distance
 
def sentence_similarity(sent1, sent2, stopwords=None):
    sent1 = sent1.split(' ')
    sent2 = sent2.split(' ')
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

In [ ]:
# df_all['search_term'] = df_all['search_term'].apply(lambda x:str_stem(x))
# df_all['product_title'] = df_all['product_title'].apply(lambda x:str_stem(x))

# df_all['product_description'] = df_all['product_description'].apply(lambda x:str_stem(x))

miss spell words correction

In [ ]:
from textblob import TextBlob
from textblob import Blobber
from textblob.sentiments import NaiveBayesAnalyzer
classifier = Blobber(analyzer=NaiveBayesAnalyzer())

def spell_correction(word):
    return classifier(word).correct()
    

df_all['search_term'] = df_all['search_term'].apply(spell_correction)
df_all['product_title'] = df_all['product_title'].apply(spell_correction)

df_all['product_description'] = df_all['product_description'].apply(spell_correction)

In [ ]:
# from textblob import Word
# df_all['search_term'] = df_all['search_term'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
# df_all['product_title'] = df_all['product_title'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
# df_all['product_description'] = df_all['product_description'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))


In [ ]:
df_all.head(3)

In [ ]:
df_all[df_all.search_term.str.count('\w+')<1]

In [ ]:
# df_all['frequency_search_product_desc'] = df_all.loc[:, ["search_term","product_description"]].apply(lambda row:str_common_word(*row), axis=1)
# df_all['frequency_search_title'] = df_all.loc[:, ["search_term","product_title"]].apply(lambda row:str_common_word(*row), axis=1)

# df_all['frequ_search_with_respect_to_sum'] = df_all.loc[:, ["frequency_search_product_desc","frequency_search_title"]].apply(lambda row:row[0]+row[1], axis=1)


In [ ]:
df_all.corr()

In [ ]:
df_all.head(3)

In [ ]:
# df_all["text_rank_sp"] = df_all.loc[:, ["search_term","product_title"]].apply(lambda x: sentence_similarity(*x), axis=1)
# df_all["text_rank_sd"] = df_all.loc[:, ["search_term","product_description"]].apply(lambda x: sentence_similarity(*x), axis=1)


In [1]:
df_all.corr()

NameError: name 'df_all' is not defined

In [ ]:
%matplotlib inline
import seaborn as sns
sns.swarmplot(x='relevance',y='text_rank_sp', data=df_all[:8000])

In [ ]:
# (df_a.product_title.str.count("\\d+") + 1).hist(bins=30)#plot number of words in title
# (training_data.search_term.str.count("\\d+") + 1).hist(bins=30) #plot number of words in search query
# (df_all.relevance.value_counts()).hist(bins=30)

# sns.pairplot(df_all[:8000])

In [ ]:
# from nltk.corpus import stopwords
# stop = stopwords.words('english')

# df_all['product_title_2'] = df_all['product_title'].apply(lambda x: len([x for x in x.split() if x in stop]))
# df_all['product_description_2'] = df_all['product_description'].apply(lambda x: len([x for x in x.split() if x in stop]))


In [ ]:
# df_all['frequency_search_product_desc2'] = df_all.loc[:, ["search_term","product_description"]].apply(lambda row:str_common_word(*row), axis=1)
# df_all['frequency_search_title2'] = df_all.loc[:, ["search_term","product_title"]].apply(lambda row:str_common_word(*row), axis=1)

# df_all['frequ_search_with_respect_to_sum2'] = df_all.loc[:, ["frequency_search_product_desc","frequency_search_title"]].apply(lambda row:row[0]+row[1], axis=1)


In [ ]:
# df_all["text_rank_sp2"] = df_all.loc[:, ["search_term","product_title"]].apply(lambda x: sentence_similarity(*x), axis=1)
# df_all["text_rank_sd2"] = df_all.loc[:, ["search_term","product_description"]].apply(lambda x: sentence_similarity(*x), axis=1)
# df_all = df_all.drop(['text_rank_sd2','text_rank_sp2','frequency_search_title2','frequency_search_product_desc2', 'frequ_search_with_respect_to_sum2','product_title_2','product_description_2'],axis=1)

In [ ]:
df_all.head(3)

In [ ]:
import seaborn as sns
sns.swarmplot(x='relevance',y='text_rank_sp', data=df_all[:8000])

In [ ]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

In [ ]:
# df_all['search_term_avg'] = df_all['search_term'].apply(lambda x: avg_word(x))
# df_all['product_description_avg'] = df_all['product_description'].apply(lambda x: avg_word(x))
# df_all['product_title_avg'] = df_all['product_title'].apply(lambda x:avg_word(x))

In [ ]:
df_all.corr()

In [ ]:
df_all[df_all.product_title_avg<1].head(3)

In [ ]:
model_data = df_all.drop(['search_term','product_title','product_description', 'frequ_search_with_respect_to_sum','distance_levistein'],axis=1)
model_data.head(3)


In [ ]:
from nltk.metrics import edit_distance
from sklearn.preprocessing import StandardScaler


In [ ]:

# df_train = model_data.iloc[:num_train]
# print('df_train',df_train)
# df_test = model_data.iloc[num_train:]
# print('df_test',df_test)
# id_test = df_test['id']

# y_train = df_train['relevance'].values
# X_train = df_train.drop(['id','relevance'],axis=1).values
# X_test = df_test.drop(['id','relevance'],axis=1).values

# #### Feature to the same scale
# scX = StandardScaler()
# X_train = scX.fit_transform(X_train)
# X_test = scX.fit_transform(X_test)

# rf = RandomForestRegressor(n_estimators=4, max_depth=6, random_state=0)
# clf = BaggingRegressor(rf, n_estimators=4, max_samples=0.1, random_state=25)
# clf.fit(X_train, y_train)
# y_pred = clf.predict(X_test)

# pd.DataFrame({"id": id_test, "relevance": y_pred}).to_csv('submission.csv',index=False)

In [ ]:
df_all.corr()

stemming


In [ ]:
import string
import re
import nltk


ps = nltk.stem.PorterStemmer()
wn = nltk.WordNetLemmatizer()

stopwords = nltk.corpus.stopwords.words('english')

def remove_punctuation(text):
    txt = ''.join(character for character in text if character not in string.punctuation)
    return txt

def tokenization(text, stopwords = None):
    tokens = re.split('\W+', text)
    if stopwords:
        tokens = [token for token in tokens if token not in stopwords]
    return tokens
        
    
def stemming(tokens):
    new_tokens = list(map(ps.stem, tokens))
#     new_tokens = [ps.stem(token) for token in tokens]
    return new_tokens

def lemmatizing(tokens):
    new_tokens = list(map(wn.lemmatize, tokens))
    return new_tokens

def result(txt):
    txt1 = remove_punctuation(txt)
    txt1 = tokenization(txt1)
    txt1 = stemming(txt1)
    return txt1
# _____________





In [2]:
# df_all['search_term_tokens'] = df_all['search_term'].apply(lambda row:result(row))
# df_all['product_description_tokens'] = df_all['product_description'].apply(lambda row:result(row))

In [ ]:
df_a